In [8]:
# FIRST CELL
import pandas as pd

# read raw data
path = "purchase_data.csv" 
raw_data_df = pd.read_csv(path, encoding="UTF-8")
raw_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [9]:
# SECOND CELL
# FIND PLAYER COUNT
# code sourced from https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe

# This code does not work - why?
total_rows = raw_data_df["SN"].nunique()
f"Total Number of Players: {total_rows}"

'Total Number of Players: 576'

In [10]:
# THIRD CELL
# FIND PURCHASING ANALYSIS TOTALS
# number of unique items
unique_items_list = raw_data_df["Item ID"].unique()
unique_items_count = len(unique_items_list)
print(f"Total Number of Unique Items: {unique_items_count}")

# Summary Table
summary_table = pd.DataFrame(
    {"Number of Unique Items": [unique_items_count],
     "Average Price": ['${0:.3}'.format(raw_data_df["Price"].mean())],
     "Number of Purchases": [raw_data_df["Price"].count()],
     "Total Revenue": [(raw_data_df["Price"].sum())]
    })
# print(summary_table)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map('${:.2f}'.format)
summary_table

# DEBUG total revenue:  .map('${:.2f}'.format)

Total Number of Unique Items: 179


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [11]:
# FOURTH CELL
# FIND GENDER DEMOGRAPHICS
# Do .unique before value_counts
gender_demo_df = pd.DataFrame(raw_data_df.groupby(["Gender"])["SN"].nunique())

print("Breakdown of Players by Gender:")
gender_demo_df

# CODE BREAKS HERE because of .format - Why???
#gender_demo_df["Percentage of Players"] = '{0:.3%}'.format(gender_demo_df["SN"] / total_rows)
gender_demo_df["Percentage of Players"] = (gender_demo_df["SN"] / total_rows).map('{:.2%}'.format)

# Formatting sourced from https://pandas.pydata.org/pandas-docs/version/0.18/generated/pandas.formats.style.Styler.format.html
#gender_demo_df.style.format({"Percentage of Players":'{0:.3%}'})
#gender_demo_df.style.format({'Percentage of Players':'{:.3%}'.format})
gender_demo_df

Breakdown of Players by Gender:


,SN,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [14]:
# FIFTH CELL
# DEBUGGING CELL
# Reformat ALL columns with floating values as Percent with 2 decimal places
# default for float is 3 decimal places
# pd.options.display.float_format = '{:.2%}'.format
#gender_demo_df

# FIND PURCHASING ANALYSIS STATS BROKEN DOWN BY GENDER
# Establish first column (Purchase Count) in the data frame
#purchase_count_by_gender["Purchase Count"] = (raw_data_df["Gender"].value_counts())
purchase_count_by_gender = pd.DataFrame(raw_data_df["Gender"].value_counts())
#purchase_count_by_gender["Gender"] = (raw_data_df["Gender"].value_counts())
purhcase_count_by_gender = purchase_count_by_gender.rename(columns={"Gender":"Purchase Count"})
purchase_count_by_gender

# Establish 2nd column (Average Purchase Price) in the data frame
gender_purchases= (raw_data_df[["Gender", "Price"]])
avg_grouped_price_by_gender = gender_purchases.groupby('Gender').mean()
purchase_count_by_gender["Average Purchase Price"] = avg_grouped_price_by_gender
purchase_count_by_gender["Average Purchase Price"] = purchase_count_by_gender["Average Purchase Price"].map('${:.2f}'.format)
purchase_count_by_gender

# Establish 3rd column (Total Purchase Value) in the data frame
sum_grouped_price_by_gender = gender_purchases.groupby('Gender').sum()
purchase_count_by_gender["Total Purchase Value"] = sum_grouped_price_by_gender
# DEBUG - Invalid Syntax
#purchase_count_by_gender["Total Purchase Value"] = purchase_count_by_gender["Total Purchase Value"].map('${:.2f}'.format)
purchase_count_by_gender

.map('${:.2f}'.format)
# Establish 4th column (Avg Total Purchase per Person) in the data frame
total_grouped_price_per_person = pd.DataFrame(raw_data_df.groupby(["SN", "Gender"]).sum())
avg_price_per_person_groued_by_gender = pd.DataFrame(total_grouped_price_per_person.groupby(["Gender"])["Price"].mean())
purchase_count_by_gender["Avg Total Purchase per Person"] = avg_price_per_person_grouped_by_gender
# DEBUG - Invalid Syntax
#purchase_count_by_gender["Avg Total Purchase per Person"] = purchase_count_by_gender["Avg Total Purchase per Person"].map('${:.2f}'.format)
purchase_count_by_gender

SyntaxError: invalid syntax (<ipython-input-14-08b32757e0aa>, line 29)

In [96]:
# SIXTH CELL
# FIND AGE DEMOGRAPHICS
age_demo_df = raw_data_df[["SN","Age"]]
age_demo_df

#GROUPING SYNTAX:  new_df = old_df.groupby(['column to group by'])

age_demo_df = raw_data_df.filter(["SN","Age", "Price"], axis=1)
no_dupe_age_demo_df = age_demo_df.drop_duplicates()

# Establish bins for ages and names for the bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
no_dupe_age_demo_df["Total Count"] = pd.cut(age_demo_df["Age"], bins, labels=group_names, include_lowest=True, right=False)

no_dupe_age_demo_df["Age Count"] = 1

# Count each bin
only_age_df = pd.DataFrame(no_dupe_age_demo_df["Total Count"].value_counts())

# Find Percentage of Players
only_age_df["Percentage of Players"] = (only_age_df["Total Count"]) / total_rows

# Reformat Percentage of Players
only_age_df["Percentage of Players"] = only_age_df["Percentage of Players"].map('{:.2%}'.format)

# Resort dataframe
only_age_df.sort_index()


#resorted_only_age_df = only_age_df.sort_index(axis=1, ascending=True)
#resorted_only_age_df

#only_age_demo_df = age_demo_df.groupby(['Age Bins'])
#only_age_demo_df
#age_demo_df.columns

#grouped_age_demo_df = age_demo_df.groupby(['SN'])
#grouped_age_demo_df


C:\Users\small\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\small\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Total Count,Percentage of Players
<10,23,3.99%
10-14,28,4.86%
15-19,135,23.44%
20-24,365,63.37%
25-29,101,17.53%
30-34,73,12.67%
35-39,41,7.12%
40+,13,2.26%


In [94]:
# SEVENTH CELL
# FIND PURCHASING STATS BY AGE BIN

# Start with all rows of purchases
age_demo_df

# Count each bin
purchase_by_age_df = pd.DataFrame(age_demo_df["Total Count"].value_counts())
purchase_by_age_df

# Resort dataframe
#purchase_by_age_df.sort_index()
#purchase_by_age_df



,Total Count
20-24,365
15-19,135
25-29,101
30-34,73
35-39,41
10-14,28
<10,23
40+,13
"[365, 135, 101, 73, 41, 28, 23, 13, 1]",1


In [ ]:
#EIGHTH CELL
# FIND PURCHASING STATS BY AGE BIN
# Create a summary data frame with SN and Count of purchases
summary_df = raw_data_df[["SN"]].unique()
summary_df
# Add to summary data frame mean purchase value

# Add to summary data frame total purchase value

# Sort dataframe decending order

# Truncate summary data frame to 1ist first 5 records